In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
pip install numpy pandas matplotlib scikit-learn tensorflow yfinance


Defaulting to user installation because normal site-packages is not writeable
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/376.0 MB 6.3 MB/s eta 0:01:00
   ---------------------------------------- 2.6/376.0 MB 7.2 MB/s eta 0:00:52
   ---------------------------------------- 4.5/376.0 MB 7.7 MB/s eta 0:00:49
    --------------------------------------- 6.8/376.0 MB 8.6 MB/s eta 0:00:44
    --------------------------------------- 9.2/376.0 MB 9.1 MB/s eta 0:00:41
   - -------------------------------------- 11.5/376.0 MB 9.5 MB/s eta 0:00:39
   - -------------------------------------- 13.9/376.0 MB 9.7 MB/s eta 0:00:38
   - -------------------------------------- 16.0/376.0 MB 9.9 MB/s eta 0:00:37
   - -------------------------------------- 18.1/376.0 MB 9.9 MB/s eta 0:00:37
   -- ------------------------------------- 20.4/376.0 MB 10.0 MB/s eta 0:00:36
  

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
def load_data(stock_symbol, start_date, end_date):
    """
    Load stock data from Yahoo Finance
    You can replace this with your own data source
    """
    import yfinance as yf
    df = yf.download(stock_symbol, start=start_date, end=end_date)
    return df

In [5]:
def preprocess_data(df, look_back=60, train_size=0.8):
    """
    Preprocess data for LSTM:
    - Use 'Close' price
    - Scale data to 0-1 range
    - Create sequences for LSTM
    - Split into train/test sets
    """
    # Use Close price
    data = df.filter(['Close']).values
    
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    
    # Create training data
    train_len = int(len(scaled_data) * train_size)
    train_data = scaled_data[0:train_len, :]
    
    # Create sequences
    X_train, y_train = [], []
    for i in range(look_back, len(train_data)):
        X_train.append(train_data[i-look_back:i, 0])
        y_train.append(train_data[i, 0])
    
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    # Create testing data
    test_data = scaled_data[train_len - look_back:, :]
    X_test, y_test = [], scaled_data[train_len:, :]
    for i in range(look_back, len(test_data)):
        X_test.append(test_data[i-look_back:i, 0])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, y_train, X_test, y_test, scaler

In [6]:

# Step 3: Build LSTM Model
def build_lstm_model(input_shape):
    """
    Build and compile LSTM model
    """
    model = Sequential()
    
    # First LSTM layer
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    
    # Second LSTM layer
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    
    # Third LSTM layer
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    
    # Output layer
    model.add(Dense(units=1))
    
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

In [7]:

# Step 4: Train Model
def train_model(model, X_train, y_train, epochs=100, batch_size=32):
    """
    Train the LSTM model with early stopping
    """
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )
    
    return model, history

In [8]:
# Step 5: Evaluate Model
def evaluate_model(model, X_test, y_test, scaler):
    """
    Make predictions and evaluate model performance
    """
    # Make predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    
    # Inverse transform actual values
    y_test = scaler.inverse_transform(y_test)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Test RMSE: {rmse:.2f}')
    
    return predictions, y_test

In [9]:
def plot_results(train, test, predictions):
    """
    Plot training data, test data, and predictions
    """
    plt.figure(figsize=(16,8))
    plt.title('Stock Price Prediction')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    
    # Plot training data
    train_len = len(train)
    plt.plot(train['Close'], label='Training Data')
    
    # Plot test data
    test_len = len(test)
    plt.plot(test.index, test['Close'], label='Actual Price')
    
    # Plot predictions
    plt.plot(test.index, predictions, label='Predicted Price')
    
    plt.legend(loc='lower right')
    plt.show()

In [10]:
if __name__ == "__main__":
    # Parameters
    STOCK_SYMBOL = 'AAPL'  # Example: Apple stock
    START_DATE = '2010-01-01'
    END_DATE = '2023-01-01'
    LOOK_BACK = 60  # Number of previous days to use for prediction
    EPOCHS = 100
    BATCH_SIZE = 32
    
    # Step 1: Load data
    df = load_data(STOCK_SYMBOL, START_DATE, END_DATE)
    
    # Step 2: Preprocess data
    X_train, y_train, X_test, y_test, scaler = preprocess_data(df, LOOK_BACK)
    
    # Step 3: Build model
    model = build_lstm_model((X_train.shape[1], 1))
    
    # Step 4: Train model
    model, history = train_model(model, X_train, y_train, EPOCHS, BATCH_SIZE)
    
    # Step 5: Evaluate model
    predictions, y_test = evaluate_model(model, X_test, y_test, scaler)
    
    # Prepare data for plotting
    train = df[:int(len(df)*0.8)]
    test = df[int(len(df)*0.8):]
    test['Predictions'] = predictions
    
    # Step 6: Plot results
    plot_results(train, test, test['Predictions'])
    
    # Optional: Save model
    model.save('stock_price_lstm.h5')

C:\Users\devad\AppData\Local\Temp\ipykernel_41620\3077839406.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stock_symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


ValueError: Found array with 0 feature(s) (shape=(3272, 0)) while a minimum of 1 is required by MinMaxScaler.